<a href="https://www.kaggle.com/code/zahravahidiferdousi/multiagent-shoppingassistant?scriptVersionId=289898869" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install requests beautifulsoup4 serpapi
!pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )
    
try:
    SERPAPI_KEY = UserSecretsClient().get_secret("SERPAPI_KEY")
    os.environ["SERPAPI_KEY"] = SERPAPI_KEY
    print("✅ Serpapi API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'SERPAPI_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.
✅ Serpapi API key setup complete.


In [4]:
# ---------------- MULTIAGENT SHOPPING ASSISTANT WITH ADK LOGGING ---------------- #

from dataclasses import dataclass
from typing import List, Dict, Any
import re
import requests
from bs4 import BeautifulSoup
import serpapi
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from google.adk.plugins import LoggingPlugin
from google.adk.tools import google_search
import nest_asyncio
import os

nest_asyncio.apply()

# ----------------------------------------------------
# 1️⃣ Data Models
# ----------------------------------------------------
@dataclass
class Money:
    amount: float
    currency: str = "EUR"

@dataclass
class Offer:
    merchant: str
    title: str
    price: Money
    availability: str
    url: str

# ----------------------------------------------------
# 2️⃣ Helper Tools
# ----------------------------------------------------
def google_search_tool(query: str, num_results: int = 5) -> List[str]:
    """Search Google for product URLs using SerpAPI."""
    params = {
        "q": query,
        "engine": "google",
        "num": num_results,
        "api_key": os.getenv("SERPAPI_KEY")
    }
    try:
        results = serpapi.search(params)
        return [res["link"] for res in results.get("organic_results", [])]
    except Exception as e:
        return []

def scrape_product_price_advanced(url: str, product_name: str) -> Dict[str, Any]:
    """Scrape product price using multiple heuristics including currency detection."""
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")

        price = None
        currency = "EUR"

        # STEP 1: Currency symbols
        # currency_regex = r"(€|\$|£|¥|CHF)\s*[\d.,]+|[\d.,]+\s*(€|\$|£|¥|CHF)"
        # matches = soup.find_all(text=re.compile(currency_regex))
        # for match in matches:
        #     text = match.strip()
        #     number = re.sub(r"[^\d.,]", "", text)
        #     if number:
        #         try:
        #             price = float(number.replace(",", "."))
        #         except:
        #             continue
        #         if "€" in text: currency = "EUR"
        #         elif "$" in text: currency = "USD"
        #         elif "£" in text: currency = "GBP"
        #         elif "CHF" in text: currency = "CHF"
        #         elif "¥" in text: currency = "JPY"
        #         break

        # STEP 2: Price classes/IDs
        if price is None:
            selectors = [
                {"attrs": {"class": re.compile(r"(price|amount|value)")}},
                {"attrs": {"id": re.compile(r"(price|amount|value)")}},
                {"attrs": {"itemprop": "price"}},
            ]
            for sel in selectors:
                tag = soup.find(attrs=sel.get("attrs"))
                if tag:
                    price_text = re.sub(r"[^\d.,]", "", tag.get_text(strip=True))
                    if price_text:
                        try:
                            price = float(price_text.replace(",", "."))
                            break
                        except:
                            continue

        # STEP 3: Fallback: number near product name
        if price is None:
            for tag in soup.find_all(text=re.compile(r"\d+[.,]?\d*")):
                parent_text = tag.parent.get_text(strip=True).lower()
                if product_name.lower() in parent_text:
                    price_text = re.sub(r"[^\d.,]", "", tag)
                    try:
                        price = float(price_text.replace(",", "."))
                        break
                    except:
                        continue

        status = "success" if price is not None else "not_found"

        return {
            "product_name": product_name,
            "price": price,
            "currency": currency,
            "url": url,
            "status": status
        }

    except Exception:
        return {
            "product_name": product_name,
            "price": None,
            "currency": "EUR",
            "url": url,
            "status": "error"
        }

def normalize_rank_tool(offers: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Sort offers by price ascending and return top 3."""
    ranked = sorted(offers, key=lambda x: x.get("price") or 999999)
    # print(ranked)
    return ranked[:3]

# ----------------------------------------------------
# 3️⃣ Define Tools
# ----------------------------------------------------
search_tool = FunctionTool(google_search_tool)
scrape_tool = FunctionTool(scrape_product_price_advanced)
rank_tool = FunctionTool(normalize_rank_tool)

# ----------------------------------------------------
# 4️⃣ Define Agents
# ----------------------------------------------------
search_agent = LlmAgent(
    name="search_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    description="Searches the web for product URLs.",
    instruction="Given a product name, use the google_search tool to get relevant URLs.",
    tools=[google_search]
)

scraper_agent = LlmAgent(
    name="scraper_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    description="Scrapes each URL for prices.",
    instruction="Given a product name and a list of URLs, call scrape_product_price_advanced for each URL.",
    tools=[scrape_tool]
)

ranking_agent = LlmAgent(
    name="ranking_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    description="Ranks offers by price.",
    instruction="Sort the offers by price ascending using rank_tool and return top 3 with URLs.",
    tools=[rank_tool]
)

# ----------------------------------------------------
# 5️⃣ Orchestrator
# ----------------------------------------------------
orchestrator = SequentialAgent(
    name="orchestrator",
    sub_agents=[search_agent, scraper_agent, ranking_agent]
)

# ----------------------------------------------------
# 6️⃣ Logging Plugin
# ----------------------------------------------------
logging_plugin = LoggingPlugin()

runner = InMemoryRunner(
    agent=orchestrator,
    plugins=[logging_plugin]  # ADK built-in logging
)

# ----------------------------------------------------
# 7️⃣ Run Example
# ----------------------------------------------------
async def run_multi(query: str):
    result = await runner.run_debug(query, quiet=True, verbose=False)
    print("\n--- FINAL RESULT ---")
    print(result[-1].content.parts[0].text)

# Usage:
await run_multi("Nike Pegasus 40 size 10")

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-f09588d7-ac00-4dfa-b592-3dca64256d51
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: orchestrator
[logging_plugin]    User Content: text: 'Nike Pegasus 40 size 10'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-f09588d7-ac00-4dfa-b592-3dca64256d51
[logging_plugin]    Starting Agent: orchestrator
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: orchestrator
[logging_plugin]    Invocation ID: e-f09588d7-ac00-4dfa-b592-3dca64256d51
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: search_agent
[logging_plugin]    Invocation ID: e-f09588d7-ac00-4dfa-b592-3dca64256d51
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: search_agent
[logging_plugin]    System Instruction: 'Given a 

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: scraper_agent
[logging_plugin]    Content: text: 'Here are the prices for the Nike Pegasus 40 in size 10 from the retailers mentioned:' | function_call: scrape_product_price_advanced | function_call: scrape_product_price_advanced | function_call: scrape_product_price_advanced | function_call: scrape_product_price_advanced
[logging_plugin]    Token Usage - Input: 261, Output: 290
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 791db05c-7d5a-41fd-85ef-eb04d4f124b8
[logging_plugin]    Author: scraper_agent
[logging_plugin]    Content: text: 'Here are the prices for the Nike Pegasus 40 in size 10 from the retailers mentioned:' | function_call: scrape_product_price_advanced | function_call: scrape_product_price_advanced | function_call: scrape_product_price_advanced | function_call: scrape_product_price_advanced
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['scrape_product_price_advanc

/tmp/ipykernel_13/410445642.py:101: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for tag in soup.find_all(text=re.compile(r"\d+[.,]?\d*")):


[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: scrape_product_price_advanced
[logging_plugin]    Agent: scraper_agent
[logging_plugin]    Function Call ID: adk-5ae26a9f-520f-45b4-9c31-69ba4ea1a50d
[logging_plugin]    Result: {'product_name': 'Nike Pegasus 40 size 10', 'price': None, 'currency': 'EUR', 'url': 'https://www.fleetfeet.com/products/nike-mens-air-zoom-pegasus-40-nkcz6794', 'status': 'not_found'}
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: scrape_product_price_advanced
[logging_plugin]    Agent: scraper_agent
[logging_plugin]    Function Call ID: adk-69511972-14cd-46a0-abad-bad9807f387e
[logging_plugin]    Arguments: {'product_name': 'Nike Pegasus 40 size 10', 'url': 'https://www.ebay.com/itm/256043191538'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: scrape_product_price_advanced
[logging_plugin]    Agent: scraper_agent
[logging_plugin]    Function Call ID: adk-69511972-14cd-46a0-abad-bad9807f387e
[logging_plugin]    

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: ranking_agent
[logging_plugin]    Content: function_call: normalize_rank_tool
[logging_plugin]    Token Usage - Input: 1013, Output: 78
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: af6e5bea-136a-4d35-982a-80c0f0a0f263
[logging_plugin]    Author: ranking_agent
[logging_plugin]    Content: function_call: normalize_rank_tool
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['normalize_rank_tool']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: normalize_rank_tool
[logging_plugin]    Agent: ranking_agent
[logging_plugin]    Function Call ID: adk-733a7a85-48a6-45b5-adec-91dfc5caf9bc
[logging_plugin]    Arguments: {'offers': [{'url': 'https://www.portlandrunningcompany.com/products/nike-mens-air-zoom-pegasus-40', 'currency': 'EUR', 'product_name': 'Nike Pegasus 40 size 10', 'price': 385.3}]}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: normalize_rank_t